In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupShuffleSplit 
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
%matplotlib inline


In [12]:
buf = np.load("../valid_files.npy", allow_pickle=True).item()
health = buf["health"]
mdd = buf["mdd"]

In [13]:
df = pd.read_csv('ml_prepared_data/entropy.csv')
df["name"] = df["name"].apply(lambda x: x + ".npy")
df = df[df["name"].isin(mdd) | df["name"].isin(health)]

In [14]:
df["patient"] = df["name"].apply(lambda x: x.split("_")[1])
del df["name"]
df.shape
df.drop(["Unnamed: 0"],axis=1, inplace=True)
df.head()

,entropy_bands_alpha_Fp1,entropy_bands_beta_Fp1,entropy_bands_gamma_Fp1,entropy_bands_theta_Fp1,entropy_bands_alpha_Fp2,entropy_bands_beta_Fp2,entropy_bands_gamma_Fp2,entropy_bands_theta_Fp2,entropy_bands_alpha_Fpz,entropy_bands_beta_Fpz,...,entropy_bands_alpha_T5,entropy_bands_beta_T5,entropy_bands_gamma_T5,entropy_bands_theta_T5,entropy_bands_alpha_T6,entropy_bands_beta_T6,entropy_bands_gamma_T6,entropy_bands_theta_T6,target,patient
0,-13.728290,-13.624056,-15.834408,-13.225958,-13.582560,-13.593790,-15.657767,-13.125128,-13.571858,-13.568165,...,-13.390705,-13.565075,-16.033698,-12.958132,-13.333594,-13.723939,-15.989141,-12.933604,1,F117
1,-12.729169,-13.587557,-15.786211,-12.958005,-12.980654,-13.779130,-16.216976,-12.792485,-12.892763,-13.563334,...,-12.555712,-13.404909,-15.817064,-12.533811,-12.014382,-13.500773,-16.125088,-12.235263,1,M163
2,-13.163862,-13.141093,-15.270868,-13.024770,-12.755880,-12.781630,-15.052215,-12.727004,-12.919386,-13.201586,...,-12.828866,-13.395307,-15.720462,-12.861393,-13.061351,-13.422073,-15.583154,-12.727259,1,F130
3,-13.232000,-13.632697,-15.843904,-12.818745,-13.363398,-13.719045,-16.058201,-12.897723,-13.235419,-13.576650,...,-13.104434,-13.705792,-15.830321,-12.835992,-13.319804,-13.728292,-15.974730,-13.106641,1,M163
4,-13.427979,-13.784442,-15.839615,-13.062695,-13.314286,-13.630312,-15.696986,-12.802822,-13.416008,-13.680343,...,-13.013987,-13.611003,-16.009145,-12.615110,-13.569824,-13.612667,-16.168686,-12.597083,1,F117


In [15]:
splitter = GroupShuffleSplit(test_size=.3, n_splits=2, random_state = 7)
split = splitter.split(df, groups=df['patient'])
train_inds, test_inds = next(split)

train = df.iloc[train_inds]
test = df.iloc[test_inds]
test["patient"].unique()
test.shape

(869, 86)

In [16]:
y_train = train["target"].values
del train["target"]
train_group = train["patient"].values
del train["patient"]

y_test = test["target"].values
del test["target"]
test_group = test["patient"].values
del test["patient"]

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

splitter_cv = GroupShuffleSplit(test_size=.2, n_splits=5, random_state = 42)
split_cv = splitter.split(train, y_train, groups=train_group)



In [9]:
from sklearn.linear_model import LogisticRegression
R = LogisticRegression(max_iter = 5000)

grid = {
    'C':np.linspace(0.001, 0.01, 100)
}

gs = GridSearchCV(R, param_grid=grid, cv=split_cv, scoring="accuracy")
gs.fit(train, y_train)

y_knc_gs = gs.predict(train)
print(accuracy_score(y_train, y_knc_gs))
y_knc_gs = gs.predict(test)
print(accuracy_score(y_test, y_knc_gs))
print(gs.best_estimator_)

0.6766398158803222
LogisticRegression(C=0.009272727272727273, max_iter=5000)


In [18]:
KN_t = KNeighborsClassifier()

grid_kn = {
    'n_neighbors':range(1, 20),
    'weights': ('uniform', 'distance'),
    'p':(1,2)
}

gs_knc = GridSearchCV(KN_t, param_grid=grid_kn, cv=split_cv, scoring="accuracy")
gs_knc.fit(train, y_train)
print(gs_knc.best_estimator_)

/home/polosatik/.local/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/home/polosatik/.local/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accept

0.6156501726121979
KNeighborsClassifier(n_neighbors=9, p=1)


/home/polosatik/.local/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/home/polosatik/.local/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accept

In [19]:
y_knc_gs = gs_knc.predict(train)
print(accuracy_score(y_train, y_knc_gs))
y_knc_gs = gs_knc.predict(test)
print(accuracy_score(y_test, y_knc_gs))


0.9868421052631579
0.6156501726121979


/home/polosatik/.local/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/home/polosatik/.local/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accept